In [6]:
from Sample import Sample
from _01_preprocess import preprocess
from _02_register import FeatureExtractor
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import tifffile as tiff
import exifread
import re
import numpy as np


def get_roi_num(f):
    with open(f, 'rb') as fin:
        tags = exifread.process_file(fin)

    roi_num = str(tags["Image Tag 0xB0B7"]).split(":")[-1].split("_")[-1].split(" ")[0]
    return roi_num

def _rmlead(inp, char='0'):
    for idx, letter in enumerate(inp):
        if letter != char:
            return inp[idx:]

In [7]:
dataset_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/BM"

In [9]:
samples = os.listdir(dataset_path)

#samples = ['20221019_19-5754_BM', '20221019_20-0568_BM']

for s in samples: 
    if not os.path.exists(os.path.join(dataset_path, s, 'IF_cut_highres')):
        os.makedirs(os.path.join(dataset_path, s, 'IF_cut_highres'))
    else:
        continue

    lowres_path = os.path.join(dataset_path, s, "IF_IMC_lowres")
    lowres_image_files = os.listdir(lowres_path)
    
    if "Thumbs.db" in lowres_image_files:
        os.remove(os.path.join(lowres_path, 'Thumbs.db'))
        lowres_image_files = os.listdir(lowres_path)

    highres_path = glob.glob(dataset_path + '/' + s + '/*[sS][pP][oO][tT][sS]*')[0]
    highres_img_files = [os.path.join(highres_path, f) for f in os.listdir(highres_path)]
    highres_img_rois = [[f,get_roi_num(f), f.split(".tif")[0][-1]] for f in highres_img_files]
    df = pd.DataFrame(highres_img_rois, columns=["file", "roi_num", "channel"])

    for f in lowres_image_files:
        lowres_roi_num = int(_rmlead(re.search('[0-9]+', f.split("_")[-3]).group()))
        lowres_img = tiff.imread(os.path.join(lowres_path, f))[-3]
        highres_img = tiff.imread(df.loc[(df['roi_num']==str(lowres_roi_num)) & (df['channel']=='B'), 'file'].values[0])

        print('Sample:' + s + ' RoI_' + str(lowres_roi_num) + ': ')
        pp_highres_img = preprocess(highres_img)
        pp_lowres_img = preprocess(lowres_img)

        ex = FeatureExtractor("sift")
        ex(pp_highres_img, pp_lowres_img)
        ex.match()
        ex.estimate()

        #Apply transformation function to all IF and IMC images
        highres_reg, lowres_reg = ex.warp(im0=highres_img, im1=lowres_img)

        tmp = np.where(lowres_reg != 0)
        highres_cut = highres_img[tmp[0].min():tmp[0].max(), tmp[1].min():tmp[1].max()]

        if lowres_roi_num <10:
            roi_prefix = "_ROI_00"
        else:
            roi_prefix = "_ROI_0"

        title= s + roi_prefix + str(lowres_roi_num) + "_IF_cut.tiff"

        tiff.imwrite(os.path.join(dataset_path, s, 'IF_cut_highres', title), highres_cut)

